In [1]:
import numpy as np
import os
from google.colab import drive
import pickle
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import torch

# deadend dependencies

In [4]:
!git clone https://github.com/microsoft/med-deadend.git


Cloning into 'med-deadend'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 52), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 395.48 KiB | 11.98 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# shaping dependencies

In [5]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 6), reused 21 (delta 4), pack-reused 0
Receiving objects: 100% (25/25), 220.90 KiB | 16.99 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
cd /content/

/content


In [7]:
%cd /content/Shaping

import zipfile

with zipfile.ZipFile('lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

/content/Shaping


In [11]:
%cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

/content/med-deadend/lifegate


In [12]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [13]:
import yaml
import random

In [14]:
from lifegate import LifeGate

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
params = yaml.safe_load(open('config.yaml', 'r'))

In [16]:
np.random.seed(seed=params['random_seed'])
random.seed(params['random_seed'])
random_state = np.random.RandomState(params['random_seed'])

In [17]:
env = LifeGate(max_steps=params['episode_max_len'], state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.4)

In [18]:
import Shaping

In [19]:
from Shaping import *

In [21]:
%cd /content/Shaping

from choose_actions import action_probs_top_n_epsilon
from shaping_features import *
from gen_policies import *
from IS import *
from subset_policies import *
from v_pi_e import *
from optimization import *
from neural_net import *
from prep_variance import *

/content/Shaping


In [22]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)

In [23]:
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

In [24]:
scope_set, phi_set = subset_policies(pi_b, 0.3)

In [25]:
IS, state_tensors, w_diff, f, st_og, psi_res, sample_last_tensors = variance_terms_tens(P_pi_e, P_pi_b, phi_set)

/content/Shaping/prep_variance.py:62: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  last_first_tens_arr = np.array(f_res)
/content/Shaping/prep_variance.py:66: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct

In [28]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
model2 = train_mse_var(model, 40, 0.001, 1, 1, IS, state_tensors, w_diff, f, sample_last_tensors, phi_set, st_og, psi_res)